# làm sạch dữ liệu

#### Import thư viện 

In [13]:
import pandas as pd
import numpy as np
import re
import datetime
import json
import ast
from pprint import pprint

### Làm sạch dữ liệu chính (sieuanhhungMarvel.csv)

#### Đọc dữ liệu thô thư file sieuanhhungMarvel.csv

In [48]:
raw_df = pd.read_csv('Data/sieuanhhungMarvel.csv')

In [49]:
raw_df.head()

,post_id,text,time,images_low,like,love,haha,sad,wow,care,angry,reaction_count,shares,comment,comments_full,year,month,day,images_count
0,734518898719448,Các bạn nam chắc ai cũng muốn học chiêu pause ...,2023-11-18 22:40:26,['https://scontent.fhan14-3.fna.fbcdn.net/v/t3...,1983,18,801,8,1,6,0,2817,21,101,"[{'comment_id': '302120192733098', 'comment_ur...",2023,11,18,2
1,734285772076094,Nếu như Pedro một mình cân tất cả các vai ở Fa...,2023-11-18 12:51:48,['https://scontent.fhan14-2.fna.fbcdn.net/v/t3...,2014,19,1667,21,4,7,1,3733,13,94,"[{'comment_id': '361135779633240', 'comment_ur...",2023,11,18,1
2,734234215414583,Sau bom tấn Spider-Man 2 thì dự án tiếp theo c...,2023-11-18 10:21:01,['https://scontent.fhan14-2.fna.fbcdn.net/v/t3...,1543,45,275,5,8,5,1,1882,9,51,"[{'comment_id': '1012246983374962', 'comment_u...",2023,11,18,1
3,734194055418599,Infinity Magic 🔥,2023-11-18 08:30:34,['https://scontent.fhan14-1.fna.fbcdn.net/v/t3...,3997,638,18,4,14,26,1,4698,34,60,"[{'comment_id': '1842669876147189', 'comment_u...",2023,11,18,1
4,733990162105655,Nam diễn viên Steven Yeun được xác nhận từ nhi...,2023-11-17 23:17:50,['https://scontent.fhan14-3.fna.fbcdn.net/v/t3...,732,56,171,1,2,3,1,966,5,56,"[{'comment_id': '1410796539783216', 'comment_u...",2023,11,17,1


#### Làm sạch dữ liệu hình ảnh từ dạng string sang dạng list

In [10]:
for i in range(len(raw_df.images_low)):
    text = raw_df.images_low.iloc[i]
    raw_df.iat[i, raw_df.columns.get_loc('images_low')] = ast.literal_eval(text)

#### Thêm cột image_count

In [12]:
image_count = []
for i in range(len(raw_df)):
    image_count.append(len(raw_df.iloc[i].images_low))

raw_df['images_count'] = image_count

raw_df.to_csv('Data/sieuanhhungMarvel.csv', index=False)

#### Thêm cột Year, Month, Day

In [40]:
# Convert 'time' to datetime format
raw_df['time'] = pd.to_datetime(raw_df['time'])

# Extract year, month, and day into separate columns
raw_df['year'] = raw_df['time'].dt.year
raw_df['month'] = raw_df['time'].dt.month
raw_df['day'] = raw_df['time'].dt.day

raw_df.to_csv('Data/sieuanhhungMarvel.csv', index=False)

In [41]:
raw_df.head()

,post_id,text,time,images_low,like,love,haha,sad,wow,care,angry,reaction_count,shares,comment,comments_full,year,month,day
0,734518898719448,Các bạn nam chắc ai cũng muốn học chiêu pause ...,2023-11-18 22:40:26,[https://scontent.fhan14-3.fna.fbcdn.net/v/t39...,1983,18,801,8,1,6,0,2817,21,101,"[{'comment_id': '302120192733098', 'comment_ur...",2023,11,18
1,734285772076094,Nếu như Pedro một mình cân tất cả các vai ở Fa...,2023-11-18 12:51:48,[https://scontent.fhan14-2.fna.fbcdn.net/v/t39...,2014,19,1667,21,4,7,1,3733,13,94,"[{'comment_id': '361135779633240', 'comment_ur...",2023,11,18
2,734234215414583,Sau bom tấn Spider-Man 2 thì dự án tiếp theo c...,2023-11-18 10:21:01,[https://scontent.fhan14-2.fna.fbcdn.net/v/t39...,1543,45,275,5,8,5,1,1882,9,51,"[{'comment_id': '1012246983374962', 'comment_u...",2023,11,18
3,734194055418599,Infinity Magic 🔥,2023-11-18 08:30:34,[https://scontent.fhan14-1.fna.fbcdn.net/v/t39...,3997,638,18,4,14,26,1,4698,34,60,"[{'comment_id': '1842669876147189', 'comment_u...",2023,11,18
4,733990162105655,Nam diễn viên Steven Yeun được xác nhận từ nhi...,2023-11-17 23:17:50,[https://scontent.fhan14-3.fna.fbcdn.net/v/t39...,732,56,171,1,2,3,1,966,5,56,"[{'comment_id': '1410796539783216', 'comment_u...",2023,11,17


#### Làm sạch comments_full (id commenters, comment time, reaction count, reactions type)

#### Lấy ra số lượng cảm xúc của comment có nhiều lượt thả reaction nhất

In [50]:
def highest_number_of_reactions_in_comments(text):
    reactions_count =[]
    pattern = r"'comment_reaction_count': (\d+)"
    matches = re.finditer(pattern, text)

    for match in matches:
        reaction_count = int(match.group(1))
        reactions_count.append(reaction_count)
    if len(reactions_count) == 0:
        return 0
    return max(reactions_count)

##### Thêm cột max_reaction_count_from_comment

In [51]:
max_reactions_count_comments = []
for i in range(len(raw_df)):
    max_reaction_count = highest_number_of_reactions_in_comments(raw_df.loc[i, 'comments_full'])
    max_reactions_count_comments.append(max_reaction_count)

raw_df['max_reaction_count_comment'] = max_reactions_count_comments

##### Lưu vào file

In [ ]:
raw_df.to_csv('Data/sieuanhhungMarvel.csv', index=False)

#### Lấy ra comment_text

In [53]:
def get_comment_text(text):
    start_marker = "'comment_text': "
    end_marker = ", 'comment_time'"
    comments = []

    # Tách chuỗi thành các phần dựa trên end_marker
    sections = text.split(end_marker)

    count = 0

    for section in sections:
        count += 1
        # Tìm vị trí của start_marker trong từng phần
        start_index = section.find(start_marker)
        
        # Nếu start_marker được tìm thấy trong phần
        if start_index != -1:
            # In ra phần sau start_marker (không bao gồm start_marker)
            result = section[start_index + len(start_marker):].strip()
            comments.append(result)
    return comments

##### Thêm cột all_comment_text

In [54]:
all_comment_texts = []
for i in range(len(raw_df)):
    text = raw_df.loc[i, 'comments_full']
    comments_text = get_comment_text(text)
    all_comment_texts.append(comments_text)

raw_df['all_comment_texts'] = all_comment_texts

##### Xóa cột comments_full

In [57]:
raw_df = raw_df.drop("comments_full", axis=1)
raw_df

,post_id,text,time,images_low,like,love,haha,sad,wow,care,angry,reaction_count,shares,comment,year,month,day,images_count,max_reaction_count_comment,all_comment_texts
0,734518898719448,Các bạn nam chắc ai cũng muốn học chiêu pause ...,2023-11-18 22:40:26,['https://scontent.fhan14-3.fna.fbcdn.net/v/t3...,1983,18,801,8,1,6,0,2817,21,101,2023,11,18,2,48,"['tôi có ý này...', 'Viet Hung ông yên tâm ai ..."
1,734285772076094,Nếu như Pedro một mình cân tất cả các vai ở Fa...,2023-11-18 12:51:48,['https://scontent.fhan14-2.fna.fbcdn.net/v/t3...,2014,19,1667,21,4,7,1,3733,13,94,2023,11,18,1,2,"['Vẫn thích John Krasinski cơ', 'Nhìn susan st..."
2,734234215414583,Sau bom tấn Spider-Man 2 thì dự án tiếp theo c...,2023-11-18 10:21:01,['https://scontent.fhan14-2.fna.fbcdn.net/v/t3...,1543,45,275,5,8,5,1,1882,9,51,2023,11,18,1,0,"['Tùng', 'Nguyễn Mạnh Cường mua PS5 đi =((']"
3,734194055418599,Infinity Magic 🔥,2023-11-18 08:30:34,['https://scontent.fhan14-1.fna.fbcdn.net/v/t3...,3997,638,18,4,14,26,1,4698,34,60,2023,11,18,1,0,"['Đỗ Thị Hồng Hạnh', 'Màu phép giống mấy viên ..."
4,733990162105655,Nam diễn viên Steven Yeun được xác nhận từ nhi...,2023-11-17 23:17:50,['https://scontent.fhan14-3.fna.fbcdn.net/v/t3...,732,56,171,1,2,3,1,966,5,56,2023,11,17,1,0,"['sentry :))', 'Trần Duy']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
211,699092455595426,So…. This it what it feels like….,2023-09-20 14:05:59,['https://scontent.fhan14-4.fna.fbcdn.net/v/t3...,4665,21,1139,1639,15,3,8,7490,118,557,2023,9,20,1,135,"['Xloi ae, tâm trạng t sau khi đăng post này k..."
212,699043455600326,Con bạn tôi: Tao không có gu bạn trai cụ thể đ...,2023-09-20 11:55:50,['https://scontent.fhan14-3.fna.fbcdn.net/v/t3...,1225,30,831,16,4,6,1,2113,10,50,2023,9,20,5,1,"['Là biến thể của nhau😂', 'Bun Bun ng có gu🥰',..."
213,698731722298166,💯💯 CỰC HOT: LẦN ĐẦU TIÊN TẠI ĐÔNG NAM Á - MARV...,2023-09-19 22:29:32,['https://scontent.fhan14-2.fna.fbcdn.net/v/t1...,167,59,1,2,4,7,1,241,8,27,2023,9,19,1,5,['Anh em Đông Lào có thể tự hào là nước đầu ti...
214,698497852321553,Một bộ phim có nhiều màu sắc bắt đầu từ những ...,2023-09-19 13:43:32,['https://scontent.fhan14-2.fna.fbcdn.net/v/t1...,1427,515,6,2,6,14,1,1971,45,86,2023,9,19,1,28,"['Xin tên nhạc', 'Phong Lêtr Masquerade - Go M..."


##### Lưu vào file

In [ ]:
raw_df.to_csv('Data/sieuanhhungMarvel.csv', index=False)

### Làm sạch file least_emotional_post_reactors.csv (reactors lấy từ post có ít lượt thả reaction nhất)

In [6]:
reactor_df = pd.read_csv('Data/least_emotional_post_reactors.csv')
reactor_df

,name,link,type
0,Minh Hieu,https://facebook.com/profile.php?id=1000917907...,like
1,Nguyễn Nhật Minh,https://facebook.com/profile.php?id=1000896896...,like
2,Trần Da-nh,https://facebook.com/profile.php?id=1000859366...,angry
3,Khánh Duy,https://facebook.com/khanh.duy.4527?eav=AfZ4al...,like
4,Đào Quang,https://facebook.com/profile.php?id=1000734822...,like
...,...,...,...
128,JG Kath,https://facebook.com/profile.php?id=6155294195...,love
129,Nguyễn Thái Ngọc,https://facebook.com/profile.php?id=6155143095...,like
130,Trần Đan Lê,https://facebook.com/profile.php?id=6155089458...,like
131,Dương Nguyễn Trọng,https://facebook.com/Duong.Hai.Phong.1991?eav=...,like


#### Lấy ra id người thả reactions

In [9]:
ids = []
for i in range(len(reactor_df)):
    if "profile.php?id" in reactor_df.loc[i, 'link']:
        id = reactor_df.loc[i, 'link'].split("profile.php?id=")[1].split("&eav")[0]
    else:
        id = reactor_df.loc[i, 'link'].split("facebook.com/")[1].split("?eav")[0]
    ids.append(id)
print(ids)

['100091790760130', '100089689677144', '100085936672198', 'khanh.duy.4527', '100073482207803', '100073452655991', 'maybachs15', '100070977557717', 'viettrance.tn', 'annguyen.hoang1204', 'owocycy', '100064554800702', '100063757544753', '100058849185281', '100058512775731', '100056606436144', 'kien.buihuu.1460', '100055769363678', 'tieu.ngoc.7393264', '100053014566847', '100047480762588', 'minhhieu.duong.77', '100044234884179', '100043411861023', 'tungson.chu', 'groot.nguyen.56', '100040130221524', '100039829983023', '100039677059273', '100039164295725', '2im.nkl', '100036754777218', 'Nolggg', '100034919262745', '100034233972434', '100033932055089', 'vu.hoanganh.334491', '100029933764120', 'moc.manh.9235', 'tuan.bap.338', '100026766336123', '100026727792459', 'haianh.hoang.9469', 'NguyenBangDos', 'babydj.ngo', 'saurs.yae', '100024013838954', '100023554389941', 'ndp.koichivas', 'tuanvu.pham.39545464', '100022639925579', 'K1oShy', '100022562713187', '100021973494198', 'tuanthuyen2402', 'cu

#### Thêm cột user id

In [10]:
reactor_df['user_id'] = ids

#### Xóa cột link

In [11]:
reactor_df = reactor_df.drop('link', axis=1)
reactor_df

,name,type,user_id
0,Minh Hieu,like,100091790760130
1,Nguyễn Nhật Minh,like,100089689677144
2,Trần Da-nh,angry,100085936672198
3,Khánh Duy,like,khanh.duy.4527
4,Đào Quang,like,100073482207803
...,...,...,...
128,JG Kath,love,61552941954842
129,Nguyễn Thái Ngọc,like,61551430958935
130,Trần Đan Lê,like,61550894587642
131,Dương Nguyễn Trọng,like,Duong.Hai.Phong.1991


#### Lưu dữ liệu đã làm sạch vào file

In [12]:
reactor_df.to_csv('Data/least_emotional_post_reactors.csv', index=False)